# Lottodata lezen om de (eventuele) winst te berekenen

Ik speel Lotto sinds 2016 en ik wil wel eens weten hoeveel geld ik daarmee verdiend heb (of, misschien, verloren heb)

## Stap1: Gegevens over de trekkingen downloaden

Door te zoeken naar "Lotto statistieken" kom ik uit op de pagina https://www.nationale-loterij.be/onze-spelen/lotto/uitslagen-trekking/statistieken . Hier kan ik de trekkingsresultaten(gamedata) en de financiële resultaten(financialdata) downloaden per jaar. Maar ik wil dit niet manueel doen. Daarom gebruik ik de *requests*-module van Python.

De data die ik download zijn *bytes*. Daarom schrijf ik de bestanden weg met 'wb' (write binary')

In [6]:
import requests

url = "https://prdlnboppreportsst.blob.core.windows.net/legal-reports/"

for jaartal in range(2016, 2025):                                  #van 2016 tot en met 2024
    gamedata_naam = f"lotto-gamedata-NL-{jaartal}.csv"
    financialdata_naam = f"lotto-financialdata-NL-{jaartal}.csv"

    gamedata = requests.get(url + gamedata_naam)
    
    with open('./data/' + gamedata_naam, "wb") as f:               #gamedata.content zijn bytes (met Byte Order Mark UTF-8)
        f.write(gamedata.content)

    financialdata = requests.get(url + financialdata_naam)

    with open('./data/' + financialdata_naam, "wb") as f:
        f.write(financialdata.content)




## Stap 2: data lezen en bewaren in een numpy array
In tegenstelling tot bij de dataframes van Pandas, moet een numpy-array data van hetzelfde type bevatten. Dat beperkt de informatie die we kunnen bewaren. Maar eerst moeten we eens kijken hoe de structuur van de twee soorten bestanden eruit ziet. 

### De structuur van gamedata

Ik gebruik numpy.genfromtxt() om eens te kijken of ik het gamedata-bestand kan lezen. De eerste rij bevat de kolomnamen (skip_header=1). De eerste kolom kan ik ook niet gebruiken omdat die de datums bevat (usecols=range(1, 8))

Omdat de lottonummers altijd positief zijn bewaren we de gegevens als unsigned integers (dtype=np.uint8)

Wanneer ik de eerste drie rijen afdruk, ziet dat er goed uit.

In [1]:
import numpy as np

jaartal = 2024
gamedata_naam = f"./data/lotto-gamedata-NL-{jaartal}.csv"

gamedata = np.genfromtxt(gamedata_naam, dtype=np.uint8, delimiter=";", skip_header=1, usecols=range(1, 8))
gamedata[:3]



array([[ 2,  3, 19, 40, 42, 43,  4],
       [21, 24, 30, 32, 39, 40,  2],
       [ 7, 11, 22, 26, 29, 42, 12]], dtype=uint8)

### De structuur van financialdata

Financialdata is wat ingewikkelder:
- ik ben alleen geïnteresseerd in de winstbedragen voor de verschillende rangen (W-Rx): usecols=range(4, 22, 2)
- de bedragen hebben als decimaalteken een komma in plaats van een punt
- daarom heb ik converters nodig (naar_float)
- om ervoor te zorgen dat de *naar_float()*-functie een string binnenkrijgt en geen bytes, moet ik ook een encoding meegeven.

In [2]:
def naar_float(tekst:str) ->float:
    return float(tekst.replace(',', '.'))

converters = { i: naar_float for i in range(4, 22, 2)}

jaartal = 2024
financialdata_naam = f"./data/lotto-financialdata-NL-{jaartal}.csv"

financialdata = np.genfromtxt(financialdata_naam, delimiter=";", skip_header=1, 
                              usecols=range(4, 22, 2), converters=converters, encoding='utf-8')


### Gamedata en Financialdata lezen

Met de glob-module van Python kan ik alle bestandsnamen opvragen die een bepaalde vorm hebben. Om de bestanden van de verschillende jaren in te lezen, voorzie ik een gewone Python list (gamedata_list/financialdata_list). Elke numpy-array wordt een item in die arrays. Tenslotte gebruik ik np.concatenate() om de arrays in de list samen te voegen.

In [1]:
import glob
import numpy as np

def naar_float(tekst:str) ->float:
    return float(tekst.replace(',', '.'))

converters = { i: naar_float for i in range(4, 22, 2)}
gamedata_bestanden = glob.glob("./data/lotto-gamedata-NL-*.csv")


gamedata_list = []
for gamedata_bestand in gamedata_bestanden:
    arr = np.genfromtxt(gamedata_bestand, dtype=np.uint8, delimiter=";", skip_header=1, usecols=range(1, 8))
    gamedata_list.append(arr)

gamedata = np.concatenate(gamedata_list)


financialdata_bestanden = glob.glob("./data/lotto-financialdata-NL-*.csv")

financialdata_list = []
for financialdata_bestand in financialdata_bestanden:
    arr = np.genfromtxt(financialdata_bestand, delimiter=";", skip_header=1, 
                        usecols=range(4, 22, 2), converters=converters, encoding='utf-8')
    financialdata_list.append(arr)

financialdata = np.concatenate(financialdata_list)


# Stap 3: de winst berekenen
## De winstrang bepalen

De functie bepaal_rang() in lotto_utils.py verwacht drie argumenten:
- een array met de 6 lottogetallen
- het bonus nummer
- een array met de 6 gespeelde getallen (mijn_nummers)

Om die functie te kunnen gebruiken voorzie ik een hulpfunctie: *controleer_cijfers()*. De bedoeling van die functie is om de rij van gamedata uit te splitsen in de 6 lottogetallen en het bonusgetal. 

np.apply_along_axis krijgt de volgende argumenten:
- de functie die moet toegepast worden op elke rij
- de as waarlangs de functie moet worden uitgevoerd (langs de kolommen)
- de array waarop de functie moet worden toegepast
- het extra argument dat we (buiten de rij) doorgeven aan de functie

In [2]:
from lotto_utils import bepaal_rang

def controleer_cijfers(rij, speel_nummers):
    lotto_nummers = rij[:6]
    bonusnummer = rij[6]
    rang = bepaal_rang(lotto_nummers, bonusnummer, speel_nummers)
    return rang

mijn_nummers = np.array([3, 9, 10, 15, 19, 452])

resultaat_rang = np.apply_along_axis(controleer_cijfers, 1, gamedata, mijn_nummers)
np.info(resultaat_rang)



class:  ndarray
shape:  (860,)
strides:  (8,)
itemsize:  8
aligned:  True
contiguous:  True
fortran:  True
data pointer: 0x27220c23df0
byteorder:  little
byteswap:  False
type: object


## De winstbedragen bepalen

### De rang toevoegen aan de array met de financialdata

Om de resultaat_rang-array te kunnen toevoegen aan de financialdata moeten er twee dingen gebeuren:
- beide arrays moeten hetzelfde aantal dimensies hebben (financialdata.shape = (860, 9) en resultaat_rang.shape=(860,))
- beide arrays moeten hetzelfde type bevatten (finacialdata bevat float64 en resultaat_rang bevat objecten)

In [3]:

financialdata_rang = np.concatenate([financialdata, resultaat_rang.astype(float).reshape(860,1)], axis=1)
print(financialdata_rang[:5])



[[5.104697e+06 5.712240e+04 1.072800e+03 2.376000e+02 2.140000e+01
  1.000000e+01 5.000000e+00 3.000000e+00 0.000000e+00          nan]
 [0.000000e+00 5.639040e+04 1.320600e+03 2.534000e+02 2.540000e+01
  1.100000e+01 5.000000e+00 3.000000e+00 0.000000e+00          nan]
 [0.000000e+00 5.385770e+04 1.571800e+03 2.646000e+02 2.900000e+01
  1.070000e+01 5.000000e+00 3.000000e+00 0.000000e+00          nan]
 [0.000000e+00 3.402430e+04 1.466900e+03 3.270000e+02 3.050000e+01
  1.260000e+01 5.000000e+00 3.000000e+00 0.000000e+00          nan]
 [0.000000e+00 4.705610e+04 2.288800e+03 4.057000e+02 3.200000e+01
  1.440000e+01 5.000000e+00 3.000000e+00 0.000000e+00 9.000000e+00]]


## Winst berekenen

Een rij in *financialdata_rang* bevat alle info die nodig is om de winst te bepalen:
- het winstbedrag voor elke rang
- de rang voor die trekking (laatste kolom)

In principe zou ik de rang nu kunnen gebruiken als index om de juiste winst te bepalen. Maar het probleem is dat die rang nu een float is geworden (en geen int). Een tweede probleem waar we rekening mee moeten houden is dat het type van de array die we terugkrijgen van np.apply_along_axis() bepaald wordt door het eerste resultaat van *bereken_winst()*. Om ervoor te zorgen dat de *tmp*-array floats bevat en geen integers, moeten we 0.0 (een float teruggeven) wanneer de rang *nan* is. 

Om de echte winst te berekenen, mag ik ook de uitgaven niet vergeten: voor elke trekking (860 op dit moment) heb ik 1,25 EUR uitgegeven.

In [4]:
def bereken_winst(rij):
    if np.isnan(rij[-1]): return 0.0
    index = int(rij[-1])
    return rij[index-1]

tmp = np.apply_along_axis(bereken_winst,1, financialdata_rang)
winst = np.sum(tmp)
print("In totaal heb ik", winst, "EUR gewonnen.")

uitgegeven = len(financialdata_rang) * 1.25

winst = winst - uitgegeven

if winst < 0:
    print("Het verlies na al die jaren Lotto spelen is",winst, "EUR.")
    print('\n"Op de Lotto spelen kan uw portemonnee schaden."')
else:
    print("Er zit een fout in de berekening.")

In totaal heb ik 482.15 EUR gewonnen.
Het verlies na al die jaren Lotto spelen is -592.85 EUR.

"Op de Lotto spelen kan uw portemonnee schaden."
